In [1]:
import re, string,os
from glob import glob as gb
import pandas as pd
from collections import Counter
from tqdm import tqdm
from utils.functions import *
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
stops = stopwords.words('english') + ["hon","member","right","friend","mr",'hon.','make','say','great']
from IPython.core.display import display, HTML

from classification import *
import spacy 
nlp = spacy.load("en_core_web_sm")

ModuleNotFoundError: No module named 'classification'

In [ ]:
from glob import glob as gb
import json 
import pandas as pd

translator = {
    "m_10":"cost",
    "m_11":"accountability",
    "m_12":"freedom",
    "m_13":"inefficiency",
    "m_14":"irrationality",
    "m_15":"centralisation",
    "m_16":"power",
    "m_8":"neutral",
    "m_9":"size"
}

lf = gb('/home/ruben/Documents/GitHub/CrisisBureaucracy/data/classifier/annotation-round-2/annotation-results/*')
ltf = gb('/home/ruben/Documents/GitHub/CrisisBureaucracy/data/classifier/annotation-round-2/annotation-texts/*')
ltf_ = {k.replace('_','-'):k for k in ltf}

In [ ]:
def load_text_labels(fn):
    try:
        with open(fn,'r') as f:
            c = json.load(f)
        labels = [translator[x] for x in list(c['metas'].keys())]

        txtn = fn[130:].replace('.ann.json','').replace('_','-')
        txtn = "/home/ruben/Documents/GitHub/CrisisBureaucracy/data/classifier/annotation-round-2/annotation-texts/" + txtn

        with open(ltf_[txtn],'r') as f:
            t = f.read()
        return [os.path.split(ltf_[txtn])[-1].replace('.txt',''),t,labels]
    except:
        print(e)
        return

In [ ]:
df = [load_text_labels(x) for x in lf]
d = []

for x in df:
    for l in x[2]:
        d.append([x[0],x[1],l])

In [ ]:
df = pd.DataFrame(d,columns=['id','text','label'])
df['text'] = utils.preprocess_(df['text'])
df['text'] = [" ".join([w.text for w in nlp(t) if w.pos_ in ["ADJ","NOUN"] and w.text not in stops]) for t in tqdm(df['text'])]

In [24]:
# Inspect Category TF-IDF terms
tfidfo, docterms = tfidf.get_docterms(df,"text",ngram_range=(1,1),max_features=2500)
tt = tfidf.get_topterms(tfidfo,docterms,df,'label')
tt = tt.head(25)
tt.head(25)
ttm = tt.melt()
unique_words = {cat:[w for w in tt[cat] if w not in set(ttm[ttm['variable'] != cat]['value'])] for cat in tt.columns}

In [39]:
cat_colors = {cat:"#"+str("%03x" % random.randint(0, 0xFFF)) for cat,v in unique_words.items()}
display(HTML(("-".join([f' <span style="background-color:{v};">{k}</span> ' for k,v in cat_colors.items()]))))
print("----------------------------------------------\n")
for c,t in enumerate(df['text'][:15]):
    print(df['label'][c])
    text = t
    for k,v in unique_words.items():
        for w in v:
            text = text.replace(f" {w} ",f' <span style="background-color:{cat_colors[k]};">{w}</span> ')
    display(HTML(text))
    print('-----------------------------------------')




----------------------------------------------

size


-----------------------------------------
size


-----------------------------------------
accountability


-----------------------------------------
inefficiency


-----------------------------------------
inefficiency


...

-----------------------------------------
freedom


-----------------------------------------
power


-----------------------------------------
neutral


-----------------------------------------
cost


-----------------------------------------


In [33]:
df

,id,text,label
0,10659-1983-07-18.15.1.4.7-conservative,new regime per cent reduction dlo work force m...,size
1,10327-1981-07-30.10.17.28.2-labour,line new clause system planning permission cou...,size
2,18728-1972-08-03.15.1.11.8-labour,modern world talk devolution responsibility pl...,accountability
3,18622-1967-02-01.8.2.88.1-conservative,order deputy speaker increase loan contain sup...,inefficiency
4,17461-1982-11-23.8.1.72.1-labour,gentleman main objective reform effective expa...,inefficiency
...,...,...,...
444,17474-1960-05-12.9.1.6.11-labour,west berlin people entitle sure moment instanc...,neutral
445,22085-1982-06-14.11.1.40.1-conservative,vast area bureaucracy existence abolish,size
446,21872-1967-05-12.7.1.4.20-ulster_unionist_party,important part british constitution many year ...,freedom
447,21872-1967-05-12.7.1.4.20-ulster_unionist_party,important part british constitution many year ...,inefficiency
